<a href="https://colab.research.google.com/github/gramirezgh/DataScienceMAE/blob/master/Copia_de_diario_OAP_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

# Nueva sección

In [ ]:
!pip install beautifulsoup4

### Import the modules

In [ ]:
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml
import json


In [ ]:
import time
from datetime import date
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 !pip install PyMySQL
 !pip install mysql-connector-python

     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 25.4MB 1.4MB/s 


In [ ]:
import pymysql

In [ ]:
#login
sessionId = 'xxxxx'
def getSessionId():
  url_login = "http://190.181.15.212:8080/precios/rest/loginService/autenticate"
  data_login = {"username":"jsuxo.dapro@gmail.com","pass":"12345"}
  headers_login =  {
      'Connection': 'keep-alive',    
      'Cache-Control': 'no-cache',
      'Accept': 'application/json, text/plain, */*',
      'Content-Type': 'application/json;charset=UTF-8',    
  }
  response_login = requests.post(url_login,data=json.dumps(data_login),headers=headers_login)
  global sessionId  
  sessionId = response_login.cookies['JSESSIONID']
  print(sessionId)



In [ ]:
getSessionId()
sessionId

7A43334648DEA0D323409A99BA07637D


'7A43334648DEA0D323409A99BA07637D'

In [ ]:
idia=1
imes=5
ianio=2021
hoy = date.today()
''' fdia=hoy.day
fmes=hoy.month
fanio=hoy.year '''
fdia=7
fmes=5
fanio=2021

In [ ]:
def getProducto(dep,prod,u_cant,u_desc):
  URL = "http://190.181.15.212:8080/precios/rest/reportService/getPrices"
  data = {"f":{"day":{"id":0,"sd":{"d":idia,"m":imes,"y":ianio},"ed":{"d":fdia,"m":fmes,"y":fanio},"a":"true"},"treg":{"prod":"true","entity":"false"}},"p":{"id":prod,"n":"Carne de Pollo entero eviscerado","ac":{"id":dep,"n":"nombre depto"},"tp":{"id":1,"n":"Mayorista"},"am":u_cant,"wa":{"id":105,"n":"Capital"},"tm":{"id":2,"n":u_desc,"m":{"id":1,"n":"PESO"}}},"b":"false"}
  headers =  {
      'Connection': 'keep-alive',    
      'Cache-Control': 'no-cache',
      'Accept': 'application/json, text/plain, */*',
      'Content-Type': 'application/json;charset=UTF-8',    
  }
  cookies = {
      'JSESSIONID': sessionId,
      'isConnect': 'true',
  }
  response = requests.post(URL, data=json.dumps(data),headers=headers,cookies=cookies,verify=False)
  if response.status_code != 200 and response.status_code != 400:
    print('Error al acceder',response.status_code)
  if response.status_code == 200:
    df = pd.DataFrame(response.json())    
    del df['dm']
    df['dep'],df['prod'] = dep,prod,
    return df
  return []


In [ ]:
resp = getProducto(2,24,100,'unid.')
resp

,d,p,dep,prod
0,2021-05-03,35.0,2,24
1,2021-05-04,35.0,2,24
2,2021-05-05,35.0,2,24
3,2021-05-06,35.0,2,24
4,2021-05-07,37.5,2,24


In [ ]:
def getUnidadDesc(s):  
  vs = s.split(' ')
  return vs[len(vs)-1]

def getUnidadCant(s):
  vs = s.split(' ')
  val = vs[len(vs)-2]
  try:
    val = int(val)
  except:
    pass
  if type(val) == int:
    return val
  else:
    return 1

diccCodOap = pd.read_excel ("/content/drive/My Drive/Colab Notebooks/data/precio/dicc_diarios_siip_oap.xlsx")
diccCodOap.dropna(inplace=True)
diccCodOap['unidad_desc'], diccCodOap['unidad_cant']= diccCodOap['unidad_oap'].apply(lambda x: getUnidadDesc(x)),diccCodOap['unidad_oap'].apply(lambda x: getUnidadCant(x))
#codOap = diccCodOap[diccCodOap['cod_oap'].notnull()]



In [ ]:
dep = (1,2,3,4,5,6,7,8,9,10080) #cob7,cbba4,alto10080,lp2,or1,potosi3,sc9,sucre5,tar6,trinid8
data = pd.DataFrame(columns=['d','p','dep'])

for index,r in diccCodOap.iterrows():
  prod= int(r['cod_oap'])
  u_cant =r['unidad_cant']
  u_desc =r['unidad_desc']  
  for d in dep:
    rows = getProducto(d,prod,u_cant,u_desc)
    if len(rows)>0:
      data = data.append(rows, ignore_index=True)


In [ ]:
data['prod']= (data['prod']).astype(int)
df1 = data.merge(diccCodOap, left_on='prod', right_on='cod_oap')
df1 = df1.replace("","NaN")
df1.dropna(inplace=True)
df1=df1[df1.p !=0]


In [ ]:
df1

,d,p,dep,prod,cod_producto,desc_producto,desc_producto_oap,unidad_oap,cod_oap,unidad_desc,unidad_cant
0,2021-05-03,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1
1,2021-05-04,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1
2,2021-05-05,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1
3,2021-05-06,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1
4,2021-05-07,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1
...,...,...,...,...,...,...,...,...,...,...,...
3038,2021-05-03,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1
3039,2021-05-04,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1
3040,2021-05-05,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1
3041,2021-05-06,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1


In [ ]:
d1 = date.today()
diccDepto = {5:1,2:2,4:3,1:4,3:5,6:6,9:7,8:8,7:9,10080:10}
df1['gestion'],df1['mes'],df1['dia'],df1['fecha_act'],df1['cod_depto'] = df1['d'].astype(str).str[0:4],(df1['d'].astype(str).str[5:7]).astype(int),(df1['d'].astype(str).str[8:10]).astype(int),d1,df1['dep'].apply(lambda x: diccDepto[x])


In [ ]:
df1

,d,p,dep,prod,cod_producto,desc_producto,desc_producto_oap,unidad_oap,cod_oap,unidad_desc,unidad_cant,gestion,mes,dia,fecha_act,cod_depto
0,2021-05-03,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1,2021,5,3,2021-05-07,4
1,2021-05-04,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1,2021,5,4,2021-05-07,4
2,2021-05-05,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1,2021,5,5,2021-05-07,4
3,2021-05-06,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1,2021,5,6,2021-05-07,4
4,2021-05-07,7.5,1,38,1,Aceite vegetal a granel nacional,Aceite vegetal a granel,Lt.,38.0,Lt.,1,2021,5,7,2021-05-07,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,2021-05-03,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1,2021,5,3,2021-05-07,10
3039,2021-05-04,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1,2021,5,4,2021-05-07,10
3040,2021-05-05,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1,2021,5,5,2021-05-07,10
3041,2021-05-06,14.5,10080,10041,39,Carne de Pollo entero eviscerado SOFIA,Carne de Pollo entero eviscerado SOFIA,Kg.,10041.0,Kg.,1,2021,5,6,2021-05-07,10


In [ ]:
df2 = df1.iloc[:,[4,7,15,11,12,13,1,14]]
columnas = ['producto','unidad','depto','gestion','mes','dia','precio','fecha_act']
df2.columns = columnas
df2


,producto,unidad,depto,gestion,mes,dia,precio,fecha_act
0,1,Lt.,4,2021,5,3,7.5,2021-05-07
1,1,Lt.,4,2021,5,4,7.5,2021-05-07
2,1,Lt.,4,2021,5,5,7.5,2021-05-07
3,1,Lt.,4,2021,5,6,7.5,2021-05-07
4,1,Lt.,4,2021,5,7,7.5,2021-05-07
...,...,...,...,...,...,...,...,...
3038,39,Kg.,10,2021,5,3,14.5,2021-05-07
3039,39,Kg.,10,2021,5,4,14.5,2021-05-07
3040,39,Kg.,10,2021,5,5,14.5,2021-05-07
3041,39,Kg.,10,2021,5,6,14.5,2021-05-07


In [ ]:
#df2.to_excel('precios_diarios_base.xlsx',index=False)

In [ ]:
#data.to_excel('precios_diarios_origen.xlsx',index=False)

In [ ]:
connection = pymysql.connect(host='190.181.4.31',
                             user='udapro',
                             password='k1cvid567',
                             db='siip')
cursor = connection.cursor()

In [ ]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in df2.columns.tolist()])

for i,row in df2.iterrows():
    sql = "INSERT INTO `sispam_diarios_prueba` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    connection.commit()